In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import math

In [ ]:
train=pd.read_table('train.tsv',sep='\\t')

# Generating Covariates

## Uni-grams

In [ ]:
from gensim import corpora, models
import nltk
import re

In [ ]:
texts = train['Phrase'].tolist()
tags=nltk.pos_tag(['word','123','12a','a123','all','the','it','this','that','me','some','small','very',
                  'not','with','black','stand','take','here','in','at','by','my',
                  'are','is','am','and','or','but','hence','to','where','when','which'])
tags

In [ ]:
# 分词过滤条件
#flags = ('n', 'nr', 'ns', 'nt', 'eng', 'v', 'd')  
stopwords = ('-',',','.',';',':',"'")  # 停词
stoptags = ['CC','CD','IN','TO','PRP','PDT','DT','PRP$','WRB'] # 词性

words_ls = []
for text in texts:
    words = [word for word in nltk.word_tokenize(text) if (word not in stopwords) and re.findall(re.compile(r'\d+'),word)==[] and (nltk.pos_tag([word])[0][1] not in stoptags)]
    words_ls.append(words)
dictionary = corpora.Dictionary(words_ls)
corpus = [dictionary.doc2bow(words) for words in words_ls]

In [ ]:
len(corpus)
len(dictionary)

In [ ]:
# with open("corpus.txt","w") as f:
#     for i in range(len(corpus)):
#         f.write(str(corpus[i])+"\n")

In [ ]:
# with open("dict.txt","w") as f:
#     for i in range(len(dictionary)):
#         f.write(dictionary[i]+"\n")

In [ ]:
dictionary=pd.read_table('dict.txt',header=None)[0].tolist()
sparse_mat=pd.read_table('corpus.txt',header=None)

In [ ]:
# sparse_mat[0]
# inds=eval(sparse_mat[0][0])
# inds

## TF-IDF

In [ ]:
sparse_mat=pd.read_table('corpus.txt',header=None)[0]

In [ ]:
corpus=[eval(sparse_mat[i]) for i in range(len(sparse_mat))]

In [ ]:
tfidf_model = models.TfidfModel(corpus)
corpus_tfidf = tfidf_model[corpus]

In [ ]:
corpus[0]

In [ ]:
corpus_tfidf[0]

In [ ]:
# with open("corpus_tfidf.txt","w") as f:
#     for i in range(len(corpus_tfidf)):
#         f.write(str(corpus_tfidf[i])+"\n")

# Regression

## Directly use all features    

In [13]:
import random

In [ ]:
data_mat=np.array(pd.read_csv("data_mat2.csv"))

y=train['Sentiment'].tolist()
y=np.array(y)
y0=(y==2)*1

x = np.concatenate((np.expand_dims(np.ones_like(data_mat[:,0],dtype=int),1),data_mat),axis=1)
del(data_mat)

### logistic

In [ ]:
run logistic.py

In [ ]:
logi=np.zeros([20,3])
i=0

In [ ]:
while i<20:
    
    train_inds=random.sample(range(len(x)),np.int(np.floor(len(x)/2)))
    test_inds=list(set(range(len(x)))-set(train_inds))

    x_train=x[train_inds,:]
    x_test=x[test_inds,:]
    y0_train=y0[train_inds]
    y0_test=y0[test_inds]
    
    w0=np.random.randn(x.shape[1])
    log_stoch_param=logistic_train(w0,x_train,y0_train,'stochastic',iter=100,epsilon=1e-5)
    log_mini_param=logistic_train(w0,x_train,y0_train,'mini',iter=100,epsilon=1e-5)
    log_full_param=logistic_train(w0,x_train,y0_train,'full',iter=100,epsilon=1e-5)
    
    logi[i,0]=logistic_valid(log_stoch_param[2],x_test,y0_test)[1]
    logi[i,1]=logistic_valid(log_mini_param[2],x_test,y0_test)[1]
    logi[i,2]=logistic_valid(log_full_param[2],x_test,y0_test)[1]
    
    print("已完成%d" % (i+1),end='\r')
    
    i+=1

In [ ]:
with open("logistic.out","a+") as f:
    for i in range(len(logi)):
        f.write(str(logi[i,0])+"\t"+str(logi[i,1])+"\t"+str(logi[i,2])+"\n")

### softmax

In [10]:
run softmax.py

Using TensorFlow backend.


In [11]:
def gen_y(y):
    
    y0=np.zeros([len(y),5],dtype=int)
    for i in range(len(y)):
        y0[i,y[i]]=1
    
    return(y0)

In [14]:
data_mat=np.array(pd.read_csv("data_mat2.csv"))
train=pd.read_table('train.tsv',sep='\t')

y=train['Sentiment'].tolist()
y=np.array(y)
y=gen_y(y)

x = np.concatenate((np.expand_dims(np.ones_like(data_mat[:,0],dtype=int),1),data_mat),axis=1)
del(data_mat)
    
train_inds=random.sample(range(len(x)),np.int(np.floor(len(x)/2)))
test_inds=list(set(range(len(x)))-set(train_inds))

x_train=x[train_inds,:]
x_test=x[test_inds,:]
y_train=y[train_inds,:]
y_test=y[test_inds,:]
    

In [16]:
w0=np.zeros([x.shape[1],5])
soft_train(w0,x_train,y_train,'stochastic',lr=0.01,iter=10,epsilon=1e-5)

(1.5982346091823456,
 0.21410568584732367,
 array([[-0.01981349, -0.00999102,  0.01982851,  0.01986846, -0.00989246],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [-0.00199599, -0.00201605,  0.00800401, -0.00199599, -0.00199599],
        ...,
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]]),
 10)

In [17]:
soft_train(w0,x_train,y_train,'mini',lr=0.01,iter=10,epsilon=1e-5)

(1.5955946825515923,
 0.5092485796061343,
 array([[-1.52841089e-02, -2.30895905e-03,  3.04837053e-02,
          9.74056841e-04, -1.38646943e-02],
        [-6.50591831e-05, -1.62659990e-06,  4.63638193e-06,
          8.73634991e-05, -2.53140981e-05],
        [-8.93747800e-04,  5.64917100e-04,  5.80116258e-04,
          4.61983809e-04, -7.13269367e-04],
        ...,
        [-5.25159060e-05, -2.17441577e-05,  5.51882632e-05,
          3.26835581e-05, -1.36117576e-05],
        [-1.87369635e-06,  5.74031771e-06,  4.16606567e-05,
         -2.54597082e-05, -2.00675698e-05],
        [-2.04342449e-06,  5.71718685e-06,  4.78005518e-06,
         -3.81066927e-06, -4.64314826e-06]]),
 10)

In [18]:
soft_train(w0,x_train,y_train,'full',lr=0.01,iter=10,epsilon=1e-5)

(1.595507772482674,
 0.5105728565936178,
 array([[-1.53857910e-02, -2.45641850e-03,  3.05206888e-02,
          1.14635358e-03, -1.38248329e-02],
        [-6.62636776e-05, -1.33572554e-05,  1.61359795e-05,
          7.62819738e-05, -1.27970204e-05],
        [-8.32518247e-04,  4.89265118e-04,  5.35325004e-04,
          5.13982937e-04, -7.06054812e-04],
        ...,
        [-5.58180546e-05, -2.34139333e-05,  8.09038440e-05,
          1.96515785e-05, -2.13234346e-05],
        [-1.89548844e-06, -2.04671013e-06,  3.90329255e-05,
         -1.76325684e-05, -1.74581585e-05],
        [-3.07297451e-06,  7.25962058e-06,  3.72985583e-06,
         -3.11251760e-06, -4.80398431e-06]]),
 10)

In [21]:
train_inds=random.sample(range(len(x)),np.int(np.floor(len(x)/2)))
test_inds=list(set(range(len(x)))-set(train_inds))
x[train_inds[:10],:].shape

(10, 333)

In [ ]:
w0=np.random.randn(x.shape[1],5)
soft_stoch_param=soft_train(w0,x_train,y_train,'stochastic',lr=0.01,iter=100,epsilon=1e-5)
soft_mini_param=soft_train(w0,x_train,y_train,'mini',lr=0.01,iter=100,epsilon=1e-5)
soft_full_param=soft_train(w0,x_train,y_train,'full',lr=0.01,iter=100,epsilon=1e-5)
    
soft[i,0]=soft_valid(soft_stoch_param[2],x_test,y_test)[1]
soft[i,1]=soft_valid(soft_mini_param[2],x_test,y_test)[1]
soft[i,2]=soft_valid(soft_full_param[2],x_test,y_test)[1]

In [ ]:
soft=np.zeros([5,3])
i=0

while i<5:
    
    train_inds=random.sample(range(len(x)),np.int(np.floor(len(x)/2)))
    test_inds=list(set(range(len(x)))-set(train_inds))

    x_train=x[train_inds,:]
    x_test=x[test_inds,:]
    y_train=y[train_inds,:]
    y_test=y[test_inds,:]
    
    w0=np.zeros([x.shape[1],5])
    soft_stoch_param=soft_train(w0,x_train,y_train,'stochastic',lr=0.01,iter=10,epsilon=1e-5)
    soft_mini_param=soft_train(w0,x_train,y_train,'mini',lr=0.01,iter=10,epsilon=1e-5)
    soft_full_param=soft_train(w0,x_train,y_train,'full',lr=0.01,iter=10,epsilon=1e-5)
    
    soft[i,0]=soft_valid(soft_stoch_param[2],x_test,y_test)[1]
    soft[i,1]=soft_valid(soft_mini_param[2],x_test,y_test)[1]
    soft[i,2]=soft_valid(soft_full_param[2],x_test,y_test)[1]
    
    print("已完成%d" % (i+1),end='\r')
    
    i+=1

In [ ]:
soft

In [ ]:
out=pd.read_table('logistic_pca5.out',header=None)

out[0].mean(),out[0].std()
out[1].mean(),out[1].std()
out[2].mean(),out[2].std()

In [ ]:
out=pd.read_table('softmax_pca3.out',header=None)

out[0].mean(),out[0].std()
out[1].mean(),out[1].std()
out[2].mean(),out[2].std()

In [8]:
out=pd.read_table('tfidf_logistic_pca3.out',header=None)

out[0].mean(),out[0].std()
out[1].mean(),out[1].std()
out[2].mean(),out[2].std()

(0.5040553633217992, 0.015276969053102453)

(0.5115828527489428, 0.012827255334056036)

(0.5099141355888761, 0.0011532802723499954)

In [23]:
out=pd.read_table('tfidf_softmax3.out',header=None)

out[0].mean(),out[0].std()
out[1].mean(),out[1].std()
out[2].mean(),out[2].std()

(0.5041484044598231, 0.04140480975443672)

(0.5100581827502243, 0.0012228038999225503)

(0.5100581827502243, 0.0012228038999225503)

In [77]:
out=pd.read_table('tfidf_softmax_pca3.out',header=None)

out[0].mean(),out[0].std()
out[1].mean(),out[1].std()
out[2].mean(),out[2].std()

(0.4970900935537613, 0.06283589312887544)

(0.5097680379341278, 0.0014200524099675963)

(0.5097680379341278, 0.0014200524099675963)

# Use Tookit

## Raw Data

In [26]:
from sklearn.linear_model import LinearRegression,LogisticRegression,Ridge,RidgeCV,Lasso, LassoCV
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,cross_validate
from sklearn import  metrics as mt

In [58]:
data_mat=np.array(pd.read_csv("data_mat2.csv"))
train=pd.read_table('train.tsv',sep='\t')

y=train['Sentiment'].tolist()
y=np.array(y)
# y=gen_y(y)
y0=(y==2)*1
# x = np.concatenate((np.expand_dims(np.ones_like(data_mat[:,0]),1),data_mat),axis=1)
x=data_mat
del(data_mat)

In [59]:
x.shape

(156060, 332)

In [88]:
test_inds=random.sample(range(len(x)),np.int(np.floor(len(x)/4)))
train_inds=list(set(range(len(x)))-set(test_inds))

x_train=x[train_inds,:]
x_test=x[test_inds,:]
y_train=y[train_inds]
y_test=y[test_inds]
y0_train=y0[train_inds]
y0_test=y0[test_inds]

In [32]:
# data_train,data_test = train_test_split(x,test_size=0.5)

# data_test.shape
# data_train.shape

In [89]:
model_soft = LogisticRegression(multi_class='multinomial',solver='sag')  
model_soft.fit(x_train, y_train)
model_log = LogisticRegression(multi_class='ovr',solver='sag')  
model_log.fit(x_train, y0_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [90]:
np.mean(model_log.predict(x_test)==y0_test)
np.mean(model_soft.predict(x_test)==y_test)

0.6710239651416122

0.5656542355504294

In [71]:
data_mat=np.array(pd.read_csv("data_mat2.csv"))
train=pd.read_table('train.tsv',sep='\t')

y=train['Sentiment'].tolist()
y=np.array(y)
# y=gen_y(y)
y0=(y==2)*1
# x = np.concatenate((np.expand_dims(np.ones_like(data_mat[:,0]),1),data_mat),axis=1)
x=data_mat
del(data_mat)

logi=[]
soft=[]

for i in range(10):
    train_inds=random.sample(range(len(x)),np.int(np.floor(len(x)/2)))
    test_inds=list(set(range(len(x)))-set(train_inds))

    x_train=x[train_inds,:]
    x_test=x[test_inds,:]
    y_train=y[train_inds]
    y_test=y[test_inds]
    y0_train=y0[train_inds]
    y0_test=y0[test_inds]
    
    model_soft = LogisticRegression(multi_class='multinomial',solver='sag')  
    model_soft.fit(x_train, y_train)
    model_log = LogisticRegression(multi_class='ovr',solver='sag')  
    model_log.fit(x_train, y0_train)
    
    logi.append(np.mean(model_log.predict(x_test)==y0_test))
    soft.append(np.mean(model_soft.predict(x_test)==y_test))

with open("tookit.raw","a+") as f:
    for i in range(10):
        f.write(str(logi[i])+"\t"+str(soft[i])+"\n")

## PCA on Raw Data

In [87]:
out=pd.read_table('toolkit.pca_tfidf',header=None)
round(out[0].mean(),4),round(out[0].std(),4)
round(out[1].mean(),4),round(out[1].std(),4)

(0.6269, 0.0015)

(0.5322, 0.0013)